<a href="https://colab.research.google.com/github/tamburins/ESAA_2023/blob/main/ESAA_5_1(497_512).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 05 감성 분석
### 감성 분석 소개
문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법으로 소셜미디어, 여론조사, 온라인 리뷰, 피드백 등 다야한 분야에서 활용되는 것으로 문서내 텍스트가 나타내는 여러 주관적인 단어와 문맥을 기비ㅏㄴ으로 감성수치를 계산하는 방법으로 긍정 감성지수와 부정 감성지수로 구성되어 긍정 감정 또는 부정 감정을 결정함

- 머신러닝 관점의 지도학습이란 학습데이터와 타깃 레이블값을 기반으로 감성분석학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성분석을 예측하는 방법으로 일반적인 텍스트 기반 분류와 유사함
- lexicon이라는 감성 어휘 사전을 이용하여 문서의 긍정 부정 감성 여부를 판단

### 지도학습 기반 감성 분석 실습 - IMDB 영화평



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd

review_df = pd.read_csv('/content/drive/MyDrive/ESAA/OB/labeledTrainData.tsv', header=0, sep="\t", quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


피치는 id(id of each data), sentiment(result of sentiment's review, target lable. 1 is positive, 0 is negative), review(text of moview review)

In [8]:
# check each text result
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

- html 형식에서 추출하여 <br/>태그가 여전히 존재하는데 이는 피처로 만들 필요가 없기 떄문에 삭제함. dataframe의 시리즈 객체에서 str을 적용하면 다양한 문자열 연산을 수행할 수 있으므로 replace를 str에 적용해 br태그를 공란으로모두 바꿈

- 그리고 영어가 아닌 숫자와 특수ㅜㄴ자 역시 sentiment를 위한 피처로는 큰 의미가 없어봉므로 이를 모두 공란으로 변경하기 위해 정규표현식 [^a-zA- Z]을 사용한다. 이는 영어 대소문자가 아닌 모든 문자를 찾는 것

In [9]:
import re

#<br> html 태그는 replace 함수로 공란으로 변환
review_df['review'] = review_df['review'].str.replace('<br/>', ' ')
#파이썬의 정규표현식 모듈인 re를 이용해 영어문자열이 아닌 문자열은 모두 공백으로 변환
review_df['review']=review_df['review'].apply(lambda x: re.sub("[^a-zA-Z]"," ", x))

In [10]:
# train test split
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id', 'sentiment'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [11]:
# feature vectorizer, classifier >
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score

# stop words > english, filtering, ngram > (1,2) 
# logisticregression c=10
pipeline=Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

# 파이프라인 객체를 이용해 핏 프레디그로 학습 예측 수행, 프레딕트 프로바는 roc_auc 때문에 수행
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


예측 정확도는 0.8865, ROC-AUC는 0.9506


In [12]:
# with tfidf vecotrization
# stop words > english, filtering, ngram > (1,2) 
# logisticregression c=10
pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

# 파이프라인 객체를 이용해 핏 프레디그로 학습 예측 수행, 프레딕트 프로바는 roc_auc 때문에 수행
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))
# better than before

예측 정확도는 0.8932, ROC-AUC는 0.9600


### 비지도학습 기반 감성분석 소개
감성만을 분석하기 위해 지원하는 감성 어휘 사전으로 한글은 지원하지 않으며 현실적으로 데이터세트가 레이블을 갖지 않을 때 유용하게 사용할 수 있다.

각각 긍정, 부정감성의 정도를 의미하는 수치를 가지며 대표적응로 nltk 패키지에서 갖는다. 단순한 의미가 아닌 문맥상의 의미를 표현하며 이러한 것을 시맨틱이라 하는데, word net은 이러한 다양한 상황에서 같은 어휘라도 다르게 사용된ㄴ 어휘의 시맨틱 정보를 제공하며, 이를 바탕으로 개별 단어를 synet이라는 개념을 이용하여 표현한다. synset은 단순 하나으 ㅣ단어가 아닌 그단어가 갖는 문맥, 시맨틱 정보를 제공하는 wordnet의 핵심 개념이다.

대신 nltk는 예측성능이 좋지 못하다는 단점이 있다.

대표적인 감성사전으로는
> SentWordNet: 감성 단어 전용의 wordnet을 구현한 것으로 wordnet의 stsnet개념을 감성분석에 적용한 것이다. sysnet별 3가지 감성점수를 할당하고 수치로 표현해 감성이 긍정인지 부정인지를 결정한다.

> VADER: 소셜미디어 텍스트에 대한 감성분석을 제공하기 위한 패키지로 대용량 텍스트 데이터에 적합하다.

> pattern: 예측 성능측면에서 가장 주목받는 패키지.

sentiwordnet, VADEr 감성사전을 이용해 감성분석을 수행한 뒤 예측 성능을 지도학습 기반의 분류와 비교해보겠다. sentiwordnet을 통해 감성사전의 전반적인 구성을 파악하고 vader를 통해 비지도학습 기반으 ㅣ감성분석을 이해하고자 한다.

### Sentiwordnet을 이용한 감성분석
#### Wordnet sysnet, sentiwordnet sentisysnet 클래스의 이해

sentiwordnet은 wordnet기반의 sysnet을 사용하므로 sysnet에 대해 이해할 필요가 있다.

In [13]:
import nltk
nltk.download('all')

from nltk.corpus import wordnet as wn
term = 'present'

synsets = wn.synsets(term)
print('synsets() 반환 type: ', type(synsets))
print('synsets() 반환 값 개수: ', len(synsets))
print('synsets() 반환 값: ', synsets)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

synsets() 반환 type:  <class 'list'>
synsets() 반환 값 개수:  18
synsets() 반환 값:  [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


반환 되는 것은 여러개의 synset 객체를 갖는 리스트로 synset('present.n.01')에서의 괄호 내용은 pos 태그를 나타내며 present는 의미, n은 명사라는 품사, 01은 그 중 첫번째 의미라는 것을 알 수 있다.

synset 객체가 갖는 여러가지 속성을 살펴보고자 한다.

In [14]:
for synset in synsets:
  print('##### Synset name : ', synset.name(), '#####')
  print('POS : ', synset.lexname())
  print('Definition:', synset.definition())
  print('Lemmas:', synset.lemma_names())

##### Synset name :  present.n.01 #####
POS :  noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
##### Synset name :  present.n.02 #####
POS :  noun.possession
Definition: something presented as a gift
Lemmas: ['present']
##### Synset name :  present.n.03 #####
POS :  noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
##### Synset name :  show.v.01 #####
POS :  verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name :  present.v.02 #####
POS :  verb.communication
Definition: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']
##### Synset name :  stage.v.01 #####
POS :  verb.creation
Definition: perform (a play), especially on a stage
Lemmas: ['stage', 'present', 're

뒤의 숫자에 따라 같은 명사지만 다른 의미를 갖는다는 것을 알 수 있으며 하나의 단어가 가질 수 있는 여러가지 시맨틱 정보를 개별 클래스로 나타낸 것이 sysnet이라는 것을 알 수 있다.

wordnet은 어떤 어휘와 다른 어휘간의 관계를 유사도로 나타낼 수 있다. synset 객체는 단어간 유사도를 나타내기 위해 path_similarity()메서드를 제공한다.

In [15]:
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree, lion, tiger, cat, dog]
similarities = []
entity_names = [entity.name().split('.')[0] for entity in entities]

#단어별 synset을 반복하면서 다른 단어의 synset과 유사도를 측정한다
for entity in entities:
  similarity = [round(entity.path_similarity(compared_entity),2)
                for compared_entity in entities]
  similarities.append(similarity)

# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 dataframe 형태로 저장
similarity_df = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


In [16]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('senti_synsets() 반환 값 개수 :', len(senti_synsets))
print('senti_synsets() 반환 값 :', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 개수 : 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [17]:
#sentisynset은 단어의 감성을 나타내는 감성지수와 객관성을 나타내는 객관성지 수를 갖는다
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수:', father.pos_score())
print('father 부정감성 지수 :', father.neg_score())
print('father 객관성 지수:', father.obj_score())
print('\n')
fabulous=swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수:', fabulous.pos_score())
print('fabulous 부정감성 지수 :', fabulous.neg_score())

father 긍정감성 지수: 0.0
father 부정감성 지수 : 0.0
father 객관성 지수: 1.0


fabulous 긍정감성 지수: 0.875
fabulous 부정감성 지수 : 0.125


### sentiwordnet을 이용한 영화 감상평 감상 분석
1 문서를 문장 단위로 분해
2 다시 문장을 단어단위로 토큰화하고 품사 태깅
3 푸사 태깅된 단어 기반의 synset 객체와 senti_synset 객체 생성
4 senti synset에서 긍정 부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정, 그렇지 않을 때 부정으로 결정



In [18]:
from nltk.corpus import wordnet as wn
# 간단한 nltk penntrebank tag 기반의 wordnet 기반 품사 tag 변환
def penn_to_wn(tag):
  if tag.startswith('J'):
    return wn.ADJ
  elif tag.startswith('N'):
    return wn.NOUN
  elif tag.startswith('R'):
    return wn.ADV
  elif tag.startswith('V'):
    return wn.VERB

In [19]:
# 문장>단어토큰>품사태깅 후 polarity score를 합산하는 함수를 생성
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
  sentiment=0.0
  tokens_count=0

  lemmatizer = WordNetLemmatizer()
  raw_sentences = sent_tokenize(text)

  for raw_sentence in raw_sentences:
    tagged_sentence = pos_tag(word_tokenize(raw_sentence))
    for word, tag in tagged_sentence:
      wn_tag = penn_to_wn(tag)
      if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        continue
      lemma = lemmatizer.lemmatize(word, pos=wn_tag)
      if not lemma:
        continue
        #어근을 추출한 단어와 wordnet 기반의 품사태깅을 입력해 synset객체 생성
        synsets = wn.synsets(lemma, pos=wn_tag)
        if not synsets:
          continue
        #sentiwordnet의 감성단어분석으로 synset 추출
        # 모든 단어에 대해 긍정감성지수는 +, 부정감성지수는 -로 합산해 감성지수 계산
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        sentiment+= (swn_synset.pos_score()-swn_synset.neg_score())
        tokens_count+= 1
    if not tokens_count:
      return 0
    # 총 score가 0이상일 경우 긍정 그렇지 않을 경우 부정
    if sentiment >=0:
      return 1
    return 0

이렇게 생성한 swn polarity 함수를 개별 문서에 적용해 긍ㅈ어 및 부정 감성을 예측해보고자 함. 지도학습 기반의 감성 분석에서 생성한 review df dataframe을 그대로 이용하고 reviewdf에 새 칼럼으로 preds를 추가해 감성평가의 결과를 담고자 함

In [24]:
review_df['preds']=review_df['review'].apply(lambda x: swn_polarity(x))
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target, preds))
print('정확도: ', np.round(accuracy_score(y_target, preds), 4))
print('정밀도: ', np.round(precision_score(y_target, preds), 4))
print('재현율: ', np.round(recall_score(y_target, preds), 4))

[[12500     0]
 [12500     0]]
정확도:  0.5
정밀도:  0.0
재현율:  0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### VADER를 이용한 감성 분석
또 다른 Lexicon인 vader lexicon을 살펴보고자 함. VADER는 소셜 미디어의 감성분석 용도로 만들어진 lexicon으로 sentimentintensityanlyzer 클래스를 이용해 쉽게 감성분석을 제공함

In [25]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_anlyzer = SentimentIntensityAnalyzer()
senti_scores = senti_anlyzer.polarity_scores(review_df['review'][0])


sentiment intensity anlyzer 객체를 새엉한 뒤에 문서별로 polarity scores 메서드를 호출해 감성점수를 구한 뒤 해당 문서의 감성점수가 임계값 이상이면 긍정 그렇지 않으면 부정으로 판단한다. sentiment intensity anlyzer 객체의 polarity scores 메서드는 딕셔너리 형태으 감성점수를 반환한다.

vader를 이용해 imdb의 감성분석을 수행한다. 이를 위해 vader polarity 함수를 새롭게 만들고자 한다. 이 함수는 입력파라미터로 영화 감상평 텍스트와 긍정부정을 결정하는 임곗값을 가지고 sentimentintensity anlyzer 객체의 polarity scores 메서드를 호출해 감성결과를 반환한다. 

In [26]:
def vader_polarity(review, threshold=0.1):
  anlyzer = SentimentIntensityAnalyzer()
  scores = anlyzer.polarity_scores(review)

  # compound 값에 기반해 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환
  agg_score =scores['compound']
  final_sentiment = 1 if agg_score>=threshold else 0
  return final_sentiment

# apply lambda 식을 이용하여 레코드별 vader polarity를 수행하고 결과를 vader preds에 저장
review_df['vader_preds']=review_df['review'].apply(lambda x : vader_polarity(x, 0.1))
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

print(confusion_matrix(y_target,vader_preds))
print('accuracy :', np.round(accuracy_score(y_target, vader_preds),4))
print('precision score :', np.round(precision_score(y_target, vader_preds),4))
print('recall_score :', np.round(recall_score(y_target, vader_preds),4))


[[ 6749  5751]
 [ 1856 10644]]
accuracy : 0.6957
precision score : 0.6492
recall_score : 0.8515
